# The following notebook is for the second assignment of the Capstone Project

## First we import the useful libraries, then we point to the wikipedia url and webscrap the page by making use of BeautifulSoup

## Then we transform the lines we want into a pandas DataFrame and reject those rows with Not assigned Borough

In [39]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki_source = requests.get(wiki_url).text

wiki_html  = BeautifulSoup(wiki_source, 'xml')
wiki_html = wiki_html.find('table')

cols = ['Postalcode', 'Borough', 'Neighborhood']
df = pd.DataFrame(columns = cols)

for tr in wiki_html.find_all('tr'):
    row_data=[]
    for td in tr.find_all('td'):
        row_data.append(td.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

mask1 = (df['Borough']!="Not assigned") 
df1 = df[mask1]

## The following is to assign the same Borough value for those postal codes with Not assigned Neighborhood

In [40]:
mask2 = df1['Neighborhood']=="Not assigned"
df1['Neighborhood'][mask2]=df1['Borough'][mask2]

C:\Users\peppe\Anaconda3\lib\site-packages\pandas\core\generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


## Let's have a look at the clean DataFrame

In [41]:
df1.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Now we merge the duplicated postalcodes in a single row

In [42]:
t1 = df1.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
t1 = t1.reset_index(drop=False)
t1.rename(columns={'Neighborhood':'Neighborhood1'}, inplace=True)

df2 = pd.merge(df1, t1, on='Postalcode')
df2.drop(['Neighborhood'], axis=1, inplace=True)
df2.drop_duplicates(inplace=True)
df2.rename(columns={'Neighborhood1':'Neighborhood'}, inplace=True)


## Let's have a look at the DataFrame

In [43]:
df2.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Last line is to get the shape of the DataFrame

In [35]:
df2.shape

(103, 3)